In [1]:
import aisuite as ai
import display_functions

In [2]:
import requests
import qrcode
from qrcode.image.styledpil import StyledPilImage
from datetime import datetime

def get_current_time():
    """
    Returns the current time as a string.
    """
    return datetime.now().strftime("%H:%M:%S")

def get_weather_from_ip():
    """
    Gets the current, high, and low temperature in Fahrenheit for the user's
    location and returns it to the user.
    """
    # Get location coordinates from the IP address
    lat, lon = requests.get('https://ipinfo.io/json').json()['loc'].split(',')

    # Set parameters for the weather API call
    params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m",
        "daily": "temperature_2m_max,temperature_2m_min",
        "temperature_unit": "fahrenheit",
        "timezone": "auto"
    }

    # Get weather data
    weather_data = requests.get("https://api.open-meteo.com/v1/forecast", params=params).json()

    # Format and return the simplified string
    return (
        f"Current: {weather_data['current']['temperature_2m']}°F, "
        f"High: {weather_data['daily']['temperature_2m_max'][0]}°F, "
        f"Low: {weather_data['daily']['temperature_2m_min'][0]}°F"
    )

# Write a text file
def write_txt_file(file_path: str, content: str):
    """
    Write a string into a .txt file (overwrites if exists).
    Args:
        file_path (str): Destination path.
        content (str): Text to write.
    Returns:
        str: Path to the written file.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)
    return file_path


# Create a QR code
def generate_qr_code(data: str, filename: str, image_path: str):
    """Generate a QR code image given data and an image path.

    Args:
        data: Text or URL to encode
        filename: Name for the output PNG file (without extension)
        image_path: Path to the image to be used in the QR code
    """
    qr = qrcode.QRCode(error_correction=qrcode.constants.ERROR_CORRECT_H)
    qr.add_data(data)

    img = qr.make_image(image_factory=StyledPilImage, embedded_image_path=image_path)
    output_file = f"{filename}.png"
    img.save(output_file)

    return f"QR code saved as {output_file} containing: {data[:50]}..."

In [15]:


def main():
    client = ai.Client(
        provider_configs={
            "ollama": {
                "api_url": "http://localhost:11434",
                "timeout": 300,
            }
        }
    )
    messages = [
        #{
        #    "role": "system", 
        #    "content": "Be concise"
        #},
        {
            "role": "user", 
            "content": "Can you tell me what time is it?"
            #"content": "Can you write me a txt note with the current weather please?"
            #"content": "Help me generate the code for data drift detection using the right library during the data preparation stage of the AI lifecycle"
        },
    ]

    ollama_llama3 = "ollama:llama3.2" 
    ollama_mistral7 = "ollama:mistral:7b" 
    #ollama_gemma = "ollama:gemma:latest"
    #ollama_deepseek_32B = "ollama:deepseek-r1:32b"

    response = client.chat.completions.create(
        model=ollama_mistral7, 
        messages=messages, 
        tools=[
        #    get_weather_from_ip,
            get_current_time,
        #    write_txt_file,
        #    generate_qr_code
        ],
        max_turns=10,
        #temperature=0.75,
    )
    print(response.choices[0].message.content)
    display_functions.pretty_print_chat_completion(response)


In [16]:


if __name__ == "__main__":
    main()